In [1]:
from pathlib import Path

# Get the current working directory as a Path object
current_path = Path.cwd()
home_folder = 'evan_home'

# Traverse up the directory tree until you find the target folder
for parent in [current_path] + list(current_path.parents):
    if parent.name == home_folder:
        home_path = parent
        break
else:
    raise ValueError(f"Folder '{home_folder}' not found in the current working directory.")

print("Home Path:", home_path)
source_code_dir = home_path / 'Source_code'
dataset_dir = home_path / 'Dataset'


Home Path: c:\Users\evanlee\Documents\Bmi_NAS_evan\evan_home


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import pickle
import xgboost as xgb

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve, average_precision_score, matthews_corrcoef
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, cross_validate, KFold, StratifiedKFold


In [4]:
import scanpy as sc

# adata = sc.read_h5ad(r"C:\Users\evanlee\Documents\Research_datasets\PBMC_Hao\GSE164378_Hao\Harmony_noZ\Hao_Harmony_test_no_scale.h5ad")
adata = sc.read_h5ad(dataset_dir / 'PBMC_Hao/GSE164378_Hao/Harmony_noZ/Hao_Harmony_test_no_scale.h5ad')
print('Original adata:', adata.shape)
adata.obs['celltype.l1'] = adata.obs['celltype.l1'].str.replace(' ', '_')
label = adata.obs['celltype.l1'].tolist()
types = np.unique(label).tolist()
print('all cell types:', types)
print('====================')
# del adata


Original adata: (161764, 33538)
all cell types: ['B', 'CD4_T', 'CD8_T', 'DC', 'Mono', 'NK', 'other', 'other_T']


In [4]:
adata.obs.head()

,celltype.l1,celltype.l2,celltype.l3,Batch,donor,time,lane,Phase,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,leiden
L1_AAACCCAAGAAACTCA,Mono,CD14 Mono,CD14 Mono,Batch1,P2,7,L1,G1,7535,217,10823,2915,4
L1_AAACCCAAGACATACA,CD4_T,CD4 TCM,CD4 TCM_1,Batch1,P1,7,L1,G1,6013,209,5864,1617,2
L1_AAACCCACAACTGGTT,CD8_T,CD8 Naive,CD8 Naive,Batch1,P4,2,L1,S,6620,213,5067,1381,5
L1_AAACCCACACGTACTA,NK,NK,NK_2,Batch1,P3,7,L1,G1,3567,202,4786,1890,3
L1_AAACCCACAGCATACT,CD8_T,CD8 Naive,CD8 Naive,Batch1,P4,7,L1,G1,6402,215,6505,1621,5


## Read features

In [5]:
import os
os.chdir(r"C:\Users\evanlee\Documents\GitHub\EvanPys\Progress\PBMC_Hao_batch_noZ\Level1\feature_selection_k3")
os.chdir(source_code_dir / 'PBMC_Hao_batch_noZ/Level1/feature_selection_k3')

features_dict = {}
# Read features for each celltype
for celltype in types:
    try:
        feature_df = pd.read_csv(f'{celltype}_features.txt', names=['Gene', 'Weight', 'Tendency'], sep='\t')
        features_dict[celltype] = feature_df
    except:
        print('skipping:', celltype)
        continue

In [6]:
features_dict.keys()

dict_keys(['B', 'CD4_T', 'CD8_T', 'DC', 'Mono', 'NK', 'other', 'other_T'])

In [7]:
count_df = pd.DataFrame(columns=['Feature_count', 'Positive_feature_count'])
for celltype in features_dict.keys():
    feature_df = features_dict[celltype]
    feature_count = feature_df.shape[0]
    positive_count = feature_df[feature_df['Tendency'] == 1].shape[0]
    count_df.loc[celltype] = [feature_count, positive_count]
count_df

,Feature_count,Positive_feature_count
B,19,10
CD4_T,201,95
CD8_T,23,9
DC,50,23
Mono,50,20
NK,33,17
other,5,3
other_T,247,112


## Test XGB classifier

In [11]:
celltype = 'B'

# subset data to celltype features
X = adata[:, features_dict[celltype]['Gene'].tolist()].X
# Binary label
y = [1 if i==celltype else 0 for i in adata.obs['celltype.l1'].tolist()]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [19]:
# Create classification matrices
# dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
# dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=False)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=False)

In [21]:
# params = {"objective": "binary:logistic", "tree_method": "gpu_hist"}  # no need to set "num_class" because binary classification
params = {"objective": "binary:logistic", "device": "cuda"}
n = 1000
model = xgb.train(
    params=params, 
    dtrain=dtrain_clf, 
    num_boost_round=n
)

In [29]:
y_prob = model.predict(dtest_clf)
y_pred = [1 if prob > 0.5 else 0 for prob in y_prob]  # Threshold at 0.5

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)
pr_auc = average_precision_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)
print("MCC:", mcc)

Accuracy: 0.9997836367570241
Precision: 0.9985512495472655
Recall: 0.9989130434782608
F1 Score: 0.9987321137475094
ROC AUC: 0.9999991429622261
PR AUC: 0.9999908163381536
MCC: 0.9986138601199674


### Cross validate: xgb.cv()

In [30]:
params = {
    "objective": "binary:logistic",
    "device": "cuda",
    "eval_metric": "logloss"
}
n = 1000

# cross validation
cv_results = xgb.cv(
   params=params, 
   dtrain=dtrain_clf, 
   num_boost_round=n,
   nfold=5,
   metrics=["logloss", "auc"],
   as_pandas=True, 
   seed=42
)
print(cv_results)

     train-logloss-mean  train-logloss-std  train-auc-mean  train-auc-std  \
0              0.166108       1.248221e-04        0.999056       0.000203   
1              0.116896       8.079640e-05        0.999504       0.000157   
2              0.084105       5.370915e-05        0.999600       0.000102   
3              0.061219       4.490751e-05        0.999705       0.000044   
4              0.044926       5.499863e-05        0.999776       0.000046   
..                  ...                ...             ...            ...   
995            0.000023       3.714273e-07        1.000000       0.000000   
996            0.000023       3.719030e-07        1.000000       0.000000   
997            0.000023       3.723933e-07        1.000000       0.000000   
998            0.000023       3.728537e-07        1.000000       0.000000   
999            0.000023       3.733349e-07        1.000000       0.000000   

     test-logloss-mean  test-logloss-std  test-auc-mean  test-auc-std  
0  

In [35]:
# Display mean AUC of cross-validation folds
mean_auc = cv_results['test-auc-mean'].iloc[-1]
print("Mean AUC across folds:", mean_auc)

Mean AUC across folds: 0.9999901886855384


### Cross validation: Stratified

In [39]:
# Stratified cross validation
params = {
    "objective": "binary:logistic",
    "device": "cuda",
    "eval_metric": "logloss", 
    'lambda': 1  # L2 regularization term on weights
}
n = 1000

k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Initialize lists to collect metrics
accuracies = []
precisions = []
recalls = []
f1_scores = []
roc_aucs = []
pr_aucs = []
mccs = []

# Perform stratified k-fold cross-validation
for train_index, test_index in skf.split(X, y):
    print(train_index, test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    
    # Create DMatrix objects for train and test sets
    dtrain_clf = xgb.DMatrix(X_train, label=y_train)
    dtest_clf = xgb.DMatrix(X_test, label=y_test)
    
    # Train the XGBoost model
    model = xgb.train(params=params, dtrain=dtrain_clf, num_boost_round=n)
    
    # Predict probabilities and classify with a 0.5 threshold
    y_prob = model.predict(dtest_clf)
    y_pred = [1 if prob > 0.5 else 0 for prob in y_prob]
    
    # Calculate and collect metrics
    accuracies.append(accuracy_score(y_test, y_pred))
    precisions.append(precision_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    roc_aucs.append(roc_auc_score(y_test, y_prob))
    pr_aucs.append(average_precision_score(y_test, y_prob))
    mccs.append(matthews_corrcoef(y_test, y_pred))

# Calculate mean and standard deviation of each metric
print("Mean Accuracy:", np.mean(accuracies), "Std Dev:", np.std(accuracies))
print("Mean Precision:", np.mean(precisions), "Std Dev:", np.std(precisions))
print("Mean Recall:", np.mean(recalls), "Std Dev:", np.std(recalls))
print("Mean F1 Score:", np.mean(f1_scores), "Std Dev:", np.std(f1_scores))
print("Mean ROC AUC:", np.mean(roc_aucs), "Std Dev:", np.std(roc_aucs))
print("Mean PR AUC:", np.mean(pr_aucs), "Std Dev:", np.std(pr_aucs))
print("Mean MCC:", np.mean(mccs), "Std Dev:", np.std(mccs))

[     0      1      2 ... 161761 161762 161763] [     3      8     16 ... 161744 161747 161751]
[     0      2      3 ... 161761 161762 161763] [     1      4      6 ... 161730 161750 161756]
[     0      1      3 ... 161760 161762 161763] [     2      5     12 ... 161749 161754 161761]
[     0      1      2 ... 161761 161762 161763] [    11     22     33 ... 161753 161757 161759]
[     1      2      3 ... 161757 161759 161761] [     0      9     10 ... 161760 161762 161763]
Mean Accuracy: 0.9996476345202616 Std Dev: 4.626306295287637e-05
Mean Precision: 0.9982607629439691 Std Dev: 0.0008376342262167467
Mean Recall: 0.9976086956521739 Std Dev: 0.0004914731871830017
Mean F1 Score: 0.9979342237366859 Std Dev: 0.00027036383628093065
Mean ROC AUC: 0.9999937313727452 Std Dev: 8.436315644889128e-06
Mean PR AUC: 0.9999127347210308 Std Dev: 0.000129920145510554
Mean MCC: 0.9977419160995993 Std Dev: 0.0002958599206462882


### Cross validation: XGBClassifier()

In [ ]:
# stratCV_results = xgb.cv(
#    params=params, 
#    dtrain=dtrain_clf, 
#    num_boost_round=n,
#    folds=skf,
#    metrics=
# )

In [8]:
celltype = 'B'

# subset data to celltype features
X = adata[:, features_dict[celltype]['Gene'].tolist()].X
# Binary label
y = [1 if i==celltype else 0 for i in adata.obs['celltype.l1'].tolist()]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [23]:
# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(n_estimators=1000, objective='binary:logistic', eval_metric='logloss', reg_lambda=1)

# Train the classifier
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [24]:
# y_prob = model.predict(dtest_clf)
# y_pred = [1 if prob > 0.5 else 0 for prob in y_prob]  # Threshold at 0.5
y_pred = xgb_clf.predict(X_test)
y_prob = xgb_clf.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)
pr_auc = average_precision_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)
print("MCC:", mcc)

Accuracy: 0.9997218186876023
Precision: 0.9981890619340819
Recall: 0.9985507246376811
F1 Score: 0.9983698605325122
ROC AUC: 0.9999989837980682
PR AUC: 0.9999890641157058
MCC: 0.9982178145523835


In [25]:
# Kfold cross validation
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1_score': 'f1',
    'roc_auc': 'roc_auc',
    'average_precision': 'average_precision',  # PR AUC
    'mcc': make_scorer(matthews_corrcoef)
}

k = 5
print('Cross validation...')
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
# cv_results = cross_validate(clf, X, y, cv=cv, scoring=scoring, n_jobs=32)
## Use training set in cross_validate()
cv_results = cross_validate(xgb_clf, X_train, y_train, cv=cv, scoring=scoring, n_jobs=32)

Cross validation...


In [26]:
cv_results

{'fit_time': array([4.62881517, 4.57806063, 4.64584613, 4.52797961, 4.60318828]),
 'score_time': array([0.59358358, 0.59966397, 0.57604027, 0.58403134, 0.58455491]),
 'test_accuracy': array([0.99957501, 0.99957499, 0.99961363, 0.99976818, 0.99972954]),
 'test_precision': array([0.99909132, 0.99863822, 0.99773551, 0.9977396 , 0.99818923]),
 'test_recall': array([0.99592391, 0.99637681, 0.99773551, 0.9995471 , 0.9986413 ]),
 'test_f1_score': array([0.9975051 , 0.99750623, 0.99773551, 0.99864253, 0.99841521]),
 'test_roc_auc': array([0.99999874, 0.99999904, 0.99999799, 0.99999049, 0.99999297]),
 'test_average_precision': array([0.99998651, 0.99998975, 0.99997839, 0.99989188, 0.99991975]),
 'test_mcc': array([0.99727433, 0.9972747 , 0.99752431, 0.99851629, 0.9982674 ])}